In [5]:
import random
import sys


# WON'T WORK EVERY TIME (cause random sampling is not consistent)
# To make it more consistent increase num_starts to a higher number, I had to keep it at 20 to make it run in under 5 minutes for the Rosalind grading system...

# Most functions can be reused from BA2F

def profile_with_pseudocounts(motifs):
    if not motifs:
        return {}
    
    k = len(motifs[0])
    t = len(motifs)
    profile = {'A': [1.0] * k, 'C': [1.0] * k, 'G': [1.0] * k, 'T': [1.0] * k} # Initialize with pseudocounts of 1, Keys are 'A', 'C', 'G', 'T'. Values are lists of probabilities for each position

    for j in range(k): # For each position
        for i in range(t): # For each motif
            nucleotide = motifs[i][j] # Motif number i for position j in the motif
            if nucleotide in profile:
                profile[nucleotide][j] += 1 #Profile[nucleotide][position] add 1

    # Normalize to get probabilities
    # Total count at each position will be t (from motifs) + 4 (from pseudocounts)
    total_count = t + 4.0 
    for nucleotide in profile:
        for j in range(k):
            profile[nucleotide][j] /= total_count
            
    return profile

def consensus(motifs):
    """Calculates the consensus string from a list of motifs."""
    if not motifs:
        return ""
    k = len(motifs[0])
    t = len(motifs)
    consensus_string = ""

    for j in range(k): # For each position
        counts = {'A': 0, 'C': 0, 'G': 0, 'T': 0}
        for i in range(t): # For each motif
            nucleotide = motifs[i][j]
            if nucleotide in counts:
                 counts[nucleotide] += 1

        # Find the most frequent nucleotide
        max_count = -1
        most_frequent_nuc = ''
        # Ensure deterministic tie-breaking (e.g., alphabetical A > C > G > T)
        for nuc in ['A', 'C', 'G', 'T']:
            if counts[nuc] > max_count:
                max_count = counts[nuc]
                most_frequent_nuc = nuc
        consensus_string += most_frequent_nuc

    return consensus_string

def score(motifs): # Sum of Hamming distances from motif to consensus
    if not motifs:
        return 0
        
    cons = consensus(motifs)
    k = len(motifs[0])
    t = len(motifs)
    total_score = 0
    
    for i in range(t): # For each motif
        motif = motifs[i]
        for j in range(k): # For each position
            if motif[j] != cons[j]:
                total_score += 1
                
    return total_score

def probability(kmer, profile): #Probability of a k-mer given a profile matrix
    prob = 1.0
    k = len(kmer)
    for j in range(k):
        nucleotide = kmer[j]
        if nucleotide in profile:
            prob *= profile[nucleotide][j]
        else:
             return 0.0 # If there is something unexpected
    return prob

def random_motifs(dna, k, t): # Select a random k-mer
    motifs = []
    for i in range(t):
        start_index = random.randint(0, len(dna[i]) - k)
        motifs.append(dna[i][start_index : start_index + k])
    return motifs

# New Function for Gibbs Sampler

def profile_randomly_generated_kmer(text, k, profile): # Selects a k-mer from text randomly, weighted by its probability
    n = len(text)
    if n < k:
        raise ValueError("Text length is less than k")

    kmers = []
    probabilities = []
    total_prob = 0.0

    for i in range(n - k + 1):
        kmer = text[i:i+k]
        prob = probability(kmer, profile)
        kmers.append(kmer)
        probabilities.append(prob)
        total_prob += prob



    chosen_kmer = random.choices(kmers, weights=probabilities, k=1)[0]
    return chosen_kmer

def gibbs_sampler(dna, k, t, N):
    # Random motifs
    current_motifs = random_motifs(dna, k, t)
    best_motifs = list(current_motifs)
    best_score = score(best_motifs)

    for j in range(N):
        i = random.randrange(t) # Choose index from 0 to t-1

        # Create profile from all motifs EXCEPT Motifs[i]
        motifs_for_profile = current_motifs[:i] + current_motifs[i+1:]

        profile = profile_with_pseudocounts(motifs_for_profile)

        # Choose a new motif for sequence i based on the profile (weighted random choice)
        current_motifs[i] = profile_randomly_generated_kmer(dna[i], k, profile)

        # Calculate score of the updated current_motifs
        current_score = score(current_motifs)

        # Update best_motifs if current score is better
        if current_score < best_score:
            best_motifs = list(current_motifs) # Make a copy
            best_score = current_score


    return best_motifs, best_score

# Function for multiple starts, 20 in this case

def run_gibbs_sampler_multiple_starts(dna, k, t, N, num_starts=20):
    overall_best_motifs = []
    overall_best_score = float('inf') # Initialize with a very high score

    print(f"Running Gibbs Sampler {num_starts} times (N={N} iterations each)...")
    for run in range(num_starts):
        # Each call to gibbs_sampler starts with new random motifs
        current_best_motifs, current_best_score = gibbs_sampler(dna, k, t, N)

        if current_best_score < overall_best_score:
            overall_best_score = current_best_score
            overall_best_motifs = current_best_motifs
            print(f"  Run {run+1}: New overall best score found: {overall_best_score}")


    print(f"Finished {num_starts} runs. Final best score: {overall_best_score}")
    return overall_best_motifs

if __name__ == "__main__":
    input_file = "../data/rosalind_ba2g.txt"
    with open(input_file, "r") as f:
        line1 = f.readline().strip().split()
        k = int(line1[0])
        t = int(line1[1])
        N = int(line1[2]) # Number of iterations per run
        dna = [line.strip() for line in f if line.strip()] # The rest of the lines are DNA codes

    best_motifs_overall = run_gibbs_sampler_multiple_starts(dna, k, t, N, num_starts=20)

    print("\nBest motifs found across all runs:")
    for motif in best_motifs_overall:
        print(motif)

Running Gibbs Sampler 20 times (N=2000 iterations each)...
  Run 1: New overall best score found: 81
  Run 2: New overall best score found: 72
  Run 4: New overall best score found: 63
Finished 20 runs. Final best score: 63

Best motifs found across all runs:
GGTACACGCAGCTCC
GCTGTTCCCATGTCA
GTGACACCCATGTCA
CCTCCACCCATGTAC
GCTCCACAGCTGTCA
GCTCCATGGATGTCA
GCTCCCTACATGTCA
GCCTTACCCATGTCA
GCTCCACCCGAATCA
GCTCCACCCATCAGA
AGCCCACCCATGTCA
GCTCGGTCCATGTCA
GCTGTCCCCATGTCA
GCTCCTGGCATGTCA
GCTCGTGCCATGTCA
AATCCACCCATGTCT
GCTCCACCCAAACCA
GCTCCACCAGGGTCA
GCTCCAGGAATGTCA
GCTCCACCCATGGAG
